### Building a Boolean-Based Model

This simple example demonstrates how to create a boolean-based model, its components, regulators, conditions and sub-conditions into [Cell Collective](https://cellcollective.org).

We'll attempt to reconstruct the [Cortical Area Development](https://research.cellcollective.org/?dashboard=true#2035:1/cortical-area-development/1) authored by CE Giacomantonio.

![](../data/images/cortical-area-development.png)

Begin by importing the ccapi module into your workspace.

In [ ]:
import ccapi

Now, let’s try creating a client object in order to interact with services provided by [Cell Collective](https://cellcollective.org).

In [ ]:
client = ccapi.Client()

Authenticate your client using a ***password flow type authentication*** scheme.

**NOTE**: *Before you can authenticate using ccapi, you must first register an application of the appropriate type on [Cell Collective](https://cellcollective.org). If you do not require a user context, it is read only.*

In [ ]:
client.auth(email = "test@cellcollective.org", password = "test")

#### Creating a Base Model

Create a Base Model using ccapi and instantize it with an authenticated client.

In [ ]:
model = ccapi.Model("Cortical Area Development", client = client)
model.save()

A `ccapi.Model` consists of various `ccapi.ModelVersion` objects that help you build various versions to a model. By default, a `ccapi.Model` provides you a default model version of a boolean type. 

In [ ]:
# get the default model version
boolean      = model.versions[0]
boolean.name = "Version 1"
boolean

#### Adding Components to a Boolean-Based Model

First, we need to create a list of component objects for this model.

In [ ]:
# create components
COUP_TFI = ccapi.InternalComponent("COUP-TFI")
EMX2     = ccapi.InternalComponent("EMX2")
FGF8     = ccapi.InternalComponent("FGF8")
PAX6     = ccapi.InternalComponent("PAX6")
Sp8      = ccapi.InternalComponent("Sp8")

Now let us add a list of components to our Boolean Model.

In [ ]:
# add components to model
boolean.add_components(COUP_TFI, EMX2, FGF8, PAX6, Sp8)

#### Saving a Model

Ensure you save your model in order to commit your work.

In [ ]:
model.save()

#### Adding Regulators, Conditions and Sub-Conditions

Let's add a list of regulators and conditions to our components. A list of regulators and conditions as well as sub-conditions can all be added at once to a component.

In [ ]:
# add regulators to components
COUP_TFI.add_regulators(
    ccapi.NegativeRegulator(Sp8),
    ccapi.NegativeRegulator(FGF8)
)
EMX2.add_regulators(
    ccapi.PositiveRegulator(COUP_TFI),
    ccapi.NegativeRegulator(FGF8),
    ccapi.NegativeRegulator(PAX6),
    ccapi.NegativeRegulator(Sp8)
)
Sp8.add_regulators(
    ccapi.PositiveRegulator(FGF8),
    ccapi.NegativeRegulator(EMX2)
)
FGF8.add_regulators(
    # add conditions to regulators
    ccapi.PositiveRegulator(FGF8, conditions = [
        ccapi.Condition(components = Sp8)
    ])
)
PAX6.add_regulators(
    ccapi.PositiveRegulator(Sp8),
    ccapi.NegativeRegulator(COUP_TFI)
)

In [ ]:
model.save()

We've now got things within our Boolean Model.

In [ ]:
boolean.components

In [ ]:
FGF8.positive_regulators

#### Model Summary

You can view detailed summary of your model using the `summary` function provided.

In [ ]:
boolean.summary()

...or view detailed information within your jupyter notebook.

In [ ]:
boolean

#### Model Rendering

You can also attempt to visualize a Boolean Model using the `draw` function provided.

In [ ]:
# boolean.draw()